In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
os.environ['NVIDIA_VISIBLE_DEVICES'] = "1"
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
os.environ


environ{'PATH': '/opt/conda/bin:/opt/conda/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin',
        'HOSTNAME': '78353253e378',
        'TERM': 'xterm-color',
        'NVIDIA_VISIBLE_DEVICES': '1',
        'CUDA_VERSION': '9.0.176',
        'CUDA_PKG_VERSION': '9-0=9.0.176-1',
        'LD_LIBRARY_PATH': '/usr/local/nvidia/lib:/usr/local/nvidia/lib64',
        'NVIDIA_DRIVER_CAPABILITIES': 'compute,utility',
        'NVIDIA_REQUIRE_CUDA': 'cuda>=9.0',
        'NCCL_VERSION': '2.4.2',
        'LIBRARY_PATH': '/usr/local/cuda/lib64/stubs',
        'CUDNN_VERSION': '7.4.2.24',
        'LANG': 'C.UTF-8',
        'HOME': '/root',
        'JPY_PARENT_PID': '6',
        'CLICOLOR': '1',
        'PAGER': 'cat',
        'GIT_PAGER': 'cat',
        'MPLBACKEND': 'module://ipykernel.pylab.backend_inline',
        'CUDA_VISIBLE_DEVICES': '1'}

In [3]:
from train import ModelLoader, load_and_save_params, MetricLoader
from datasets.cvpr2016_cub_loader import Cvpr2016CubLoader
from common.util import Namespace
import numpy as np
from tqdm import tqdm
from scipy.spatial import cKDTree, KDTree
import time
from common.metrics import ap_at_k_prototypes
import os, pwd
from common.pretrained_models import InceptionV3Loader, InceptionV2Loader
from common.pretrained_models import INCEPTION_V2_PATH
from PIL import Image
from datasets.dataset_list import get_dataset_splits
from common.metrics import get_prototypes, top1_gzsl

Using TensorFlow backend.


## Test nearest neighbour model loader

In [4]:
alpha = 0.02

In [5]:
model_dir = 'mi_weight=0.5;repeat=0;txt2img_weight=0.5;test_split=test;train_split=trainval;dataset=xian2017_cub;text_feature_extractor=cnn_bi_lstm;number_of_steps=150001;modality_interaction=NONE'
# model_dir = 'mi_weight=0.5;repeat=0;txt2img_weight=0.5;test_split=test;train_split=trainval;dataset=xian2018_flowers;text_feature_extractor=cnn_bi_lstm;number_of_steps=150001;modality_interaction=NONE'

log_dir = f"/mnt/scratch/boris/experiments_zeroshot_pairwise/190513_022122_txt2img_weight_dataset_text_feature_extractor_number_of_steps_repeat_modality_interaction_mi_weight_test_split_train_split_GZSL_results_test_split/{model_dir}"


metric_model = MetricLoader(model_path=log_dir, batch_size_image=100, batch_size_text=200)

feature_model = ModelLoader(model_path=log_dir, batch_size=None,
                            num_images=metric_model.flags.num_images, num_texts=metric_model.flags.num_texts, 
                            max_text_len=30)


INFO:root:Searching for '/mnt/scratch/boris/experiments_zeroshot_pairwise/190513_022122_txt2img_weight_dataset_text_feature_extractor_number_of_steps_repeat_modality_interaction_mi_weight_test_split_train_split_GZSL_results_test_split/mi_weight=0.5;repeat=0;txt2img_weight=0.5;test_split=test;train_split=trainval;dataset=xian2017_cub;text_feature_extractor=cnn_bi_lstm;number_of_steps=150001;modality_interaction=NONE/params.json'
INFO:root:Loading existing params.
INFO:root:Searching for '/mnt/scratch/boris/experiments_zeroshot_pairwise/190513_022122_txt2img_weight_dataset_text_feature_extractor_number_of_steps_repeat_modality_interaction_mi_weight_test_split_train_split_GZSL_results_test_split/mi_weight=0.5;repeat=0;txt2img_weight=0.5;test_split=test;train_split=trainval;dataset=xian2017_cub;text_feature_extractor=cnn_bi_lstm;number_of_steps=150001;modality_interaction=NONE/params.json'
INFO:root:Loading existing params.


INFO:tensorflow:Restoring parameters from /mnt/scratch/boris/experiments_zeroshot_pairwise/190513_022122_txt2img_weight_dataset_text_feature_extractor_number_of_steps_repeat_modality_interaction_mi_weight_test_split_train_split_GZSL_results_test_split/mi_weight=0.5;repeat=0;txt2img_weight=0.5;test_split=test;train_split=trainval;dataset=xian2017_cub;text_feature_extractor=cnn_bi_lstm;number_of_steps=150001;modality_interaction=NONE/train/model-150000


INFO:tensorflow:Restoring parameters from /mnt/scratch/boris/experiments_zeroshot_pairwise/190513_022122_txt2img_weight_dataset_text_feature_extractor_number_of_steps_repeat_modality_interaction_mi_weight_test_split_train_split_GZSL_results_test_split/mi_weight=0.5;repeat=0;txt2img_weight=0.5;test_split=test;train_split=trainval;dataset=xian2017_cub;text_feature_extractor=cnn_bi_lstm;number_of_steps=150001;modality_interaction=NONE/train/model-150000


In [6]:
dataset_splits = get_dataset_splits(dataset_name=metric_model.flags.dataset, data_dir=metric_model.flags.data_dir,
                                    splits=[metric_model.flags.train_split, 
                                            metric_model.flags.test_split+"_seen", 
                                            metric_model.flags.test_split+"_unseen"], 
                                    flags=metric_model.flags)

INFO:root:Loading split trainval
INFO:root:Loading cached file /mnt/datasets/public/research/cvpr2016_cub/split_trainval_xian2017.pkl
INFO:root:Loaded cache in 8.074689 sec
INFO:root:Loading split test_seen
INFO:root:Loading cached file /mnt/datasets/public/research/cvpr2016_cub/split_test_seen_xian2017.pkl
INFO:root:Loaded cache in 2.866140 sec
INFO:root:Loading split test_unseen
INFO:root:Loading cached file /mnt/datasets/public/research/cvpr2016_cub/split_test_unseen_xian2017.pkl
INFO:root:Loaded cache in 6.835209 sec


In [7]:
results_eval, embeddings = feature_model.eval_acc_gzsh(train_loader=dataset_splits[metric_model.flags.train_split], 
                                                       test_loader_seen=dataset_splits[metric_model.flags.test_split+"_seen"],
                                                       test_loader_unseen=dataset_splits[metric_model.flags.test_split+"_unseen"],
                                                       batch_size=32, seen_adjustment=0.02) # 0.015
results_eval



INFO:root:Computing train embeddings
221it [00:07, 27.71it/s]
INFO:root:Computing test embeddings, unseen
93it [00:02, 39.11it/s]
INFO:root:Computing test embeddings, seen
56it [00:01, 31.26it/s]
INFO:root:Computing generalized zero-shot performance metrics
INFO:root:Searching for '/mnt/scratch/boris/experiments_zeroshot_pairwise/190513_022122_txt2img_weight_dataset_text_feature_extractor_number_of_steps_repeat_modality_interaction_mi_weight_test_split_train_split_GZSL_results_test_split/mi_weight=0.5;repeat=0;txt2img_weight=0.5;test_split=test;train_split=trainval;dataset=xian2017_cub;text_feature_extractor=cnn_bi_lstm;number_of_steps=150001;modality_interaction=NONE/params.json'
INFO:root:Loading existing params.
100%|██████████| 6/6 [00:00<00:00,  7.91it/s]
INFO:root:Computing classical zero-shot performance metrics, test
INFO:root:Searching for '/mnt/scratch/boris/experiments_zeroshot_pairwise/190513_022122_txt2img_weight_dataset_text_feature_extractor_number_of_steps_repeat_modali

{'train_AP@50/#sentences10': 0.27546666666666669,
 'train_Top-1 Acc/#sentences10': 0.5038718115957046,
 'train_AP@50/#sentences50': 0.33493333333333331,
 'train_Top-1 Acc/#sentences50': 0.73742134804553949,
 'train_AP@50/#sentences100': 0.35693333333333338,
 'train_Top-1 Acc/#sentences100': 0.78107854798920373,
 'train_AP@50/#sentences200': 0.35693333333333338,
 'train_Top-1 Acc/#sentences200': 0.81136396747495765,
 'train_AP@50/#sentences400': 0.35813333333333336,
 'train_Top-1 Acc/#sentences400': 0.82568059673903593,
 'train_AP@50/#sentences1000': 0.35880000000000006,
 'train_Top-1 Acc/#sentences1000': 0.82647587532851052,
 'test_AP@50/#sentences10': 0.32879999999999998,
 'test_Top-1 Acc/#sentences10': 0.45823161541802082,
 'test_AP@50/#sentences50': 0.36199999999999993,
 'test_Top-1 Acc/#sentences50': 0.595602774027578,
 'test_AP@50/#sentences100': 0.35920000000000002,
 'test_Top-1 Acc/#sentences100': 0.65727380015049819,
 'test_AP@50/#sentences200': 0.36920000000000008,
 'test_Top-

# Reimplement the evaluation framework and verify it

### Compute embeddings

In [8]:
image_embeddings_train, text_embeddings_train = feature_model.predict_all(dataset_splits["trainval"], batch_size=100)

image_embeddings_seen, text_embeddings_seen = feature_model.predict_all(dataset_splits["test_seen"], batch_size=100)

image_embeddings_unseen, text_embeddings_unseen = feature_model.predict_all(dataset_splits["test_unseen"], batch_size=100)



71it [00:04, 13.11it/s]
18it [00:01, 13.21it/s]
30it [00:02, 12.28it/s]


### Compute class prototypes

In [9]:
labels_train = np.array(dataset_splits["trainval"].image_classes)
labels_seen = np.array(dataset_splits["test_seen"].image_classes)
labels_unseen = np.array(dataset_splits["test_unseen"].image_classes)
labels_all = np.concatenate([labels_train, labels_seen, labels_unseen])

classes_seen = np.array(list(set(labels_train)))
classes_unseen = np.array(list(set(labels_unseen)))

text_embeddings_all = np.concatenate([text_embeddings_train, text_embeddings_seen, text_embeddings_unseen])
seen_unseen_labels_indicator = np.isin(np.array(labels_all), classes_seen)


def get_prototypes(embeddings, labels):
    class_labels = list(set(labels))
    labels = np.array(labels)
    prototypes = dict()
    for c in class_labels:
        prototypes[c] = embeddings[labels == c].mean(axis=0)
    return prototypes
    
    
prototypes_dict = get_prototypes(embeddings=text_embeddings_all, labels=labels_all)

prototypes = np.array(list(prototypes_dict.values()))
prototypes_classes = np.array(list(prototypes_dict.keys()))
seen_unseen_prototypes = np.isin(prototypes_classes, classes_seen)

assert len(labels_all) == len(labels_train) + len(labels_seen) + len(labels_unseen)
# assert len(prototypes_dict) == 200
assert len(prototypes_dict['001']) == 1024
assert sum(seen_unseen_labels_indicator) == len(labels_train) + len(labels_seen)
assert sum(~seen_unseen_labels_indicator) == len(labels_unseen)
assert sum(seen_unseen_prototypes) == len(classes_seen)



### Define the Top-1 accuracy metric

In [10]:
def top1_acc(labels, predictions):
    class_labels = list(set(labels))
    match_count = dict()
    total_count = dict()
    acc_by_class = dict()
    for c in class_labels:
        entries_of_class_c = labels == c
        total_count[c] = sum(entries_of_class_c)
        match_count[c] = sum(labels[entries_of_class_c] == predictions[entries_of_class_c])
        acc_by_class[c] = match_count[c] / total_count[c]
    
    return np.array(list(acc_by_class.values())).mean()
    
assert top1_acc(labels=labels_all, predictions=labels_all) == 1.0

print("Accuracy of random permutation")
top1_acc(labels=labels_all, predictions=np.random.choice(labels_all, size=len(labels_all), replace=False))


Accuracy of random permutation


0.0056854557057210812

### Implement nearest prototype classifier

In [11]:
def nearest_prototype_classify(prototypes_dict, seen_unseen_prototypes, query_embeddings, alpha):
    prototypes = np.array(list(prototypes_dict.values()))
    prototypes_classes = np.array(list(prototypes_dict.keys()))
    
    dist = np.sqrt(np.square(prototypes[:,:,None] - query_embeddings.transpose()).sum(axis=-2))
    dist = dist * (1.0 + alpha*seen_unseen_prototypes[:,None])
    
    return prototypes_classes[np.argmin(dist, axis=0)]


In [12]:
predictions_seen = nearest_prototype_classify(prototypes_dict = prototypes_dict, 
                                               seen_unseen_prototypes = seen_unseen_prototypes, 
                                               query_embeddings = image_embeddings_seen, 
                                               alpha = alpha)
predictions_unseen = nearest_prototype_classify(prototypes_dict = prototypes_dict, 
                                               seen_unseen_prototypes = seen_unseen_prototypes, 
                                               query_embeddings = image_embeddings_unseen, 
                                               alpha = alpha)


In [13]:
s = top1_acc(labels=labels_seen, predictions=predictions_seen)
u = top1_acc(labels=labels_unseen, predictions=predictions_unseen)
H = 2*s*u / (s+u)


In [14]:
print("Accuracy seen:", s)
print("Accuracy unseen:", u)
print("Accuracy harmonic mean:", H)


Accuracy seen: 0.527900400797
Accuracy unseen: 0.604925784903
Accuracy harmonic mean: 0.563794460852
